## Import Custom Quant Engine Wrapper

In [1]:
import myqengine as qe

## TimeSeries

In [2]:
time_series = qe.TimeSeries(start_date='-1M',
                            end_date = '0M',
                            frequency='M',
                            calendar="NAY")
time_series.get_dates()

{'calendar': 'NAY',
 'end_date': '0M',
 'frequency': 'M',
 'source': 'FdsDate',
 'start_date': '-1M'}

## Universe
### Define universe using a Screening Expression

In [3]:
#Define a universe object including all constituents of the S&P 500 as of latest close
univ = qe.ScreenUniverse(universe_expr = 'FG_CONSTITUENTS(SP50,0,CLOSE)',
                    universe_type ='Equity')
univ.get_univ()

{'security_expr': 'TICKER',
 'source': 'ScreeningExpressionUniverse',
 'universe_expr': 'FG_CONSTITUENTS(SP50,0,CLOSE)',
 'universe_type': 'Equity'}

### Define Universe using a list of IDs

In [4]:
id_univ = qe.IdUniverse(ids = ['NVDA-US','AAPL-US','IVV-US'],universe_type='Equity')
id_univ.get_univ()

{'identifiers': ['NVDA-US', 'AAPL-US', 'IVV-US'],
 'source': 'IdentifierUniverse',
 'universe_type': 'Equity'}

## Get Data

### Calculate formulas using screening

In [6]:
### Generate Data using Screening
formulas = {'price':'P_PRICE(0,USD)',
            'sector':'GICS_SECTOR'}
df = qe.calculate(universe=univ, data_dict=formulas,dates = time_series)
df.data

,DATE,UNIVERSE,price,sector
0,20230731,CTAS,451.62,NaN
1,20230731,RTX,100.92,NaN
2,20230731,WEC,93.76,NaN
3,20230731,MAA,156.99,NaN
4,20230731,AES,28.76,NaN
...,...,...,...,...
1001,20230831,OTIS,78.31,NaN
1002,20230831,MTCH,41.49,NaN
1003,20230831,GEHC,58.38,NaN
1004,20230831,WBD,9.48,NaN


### FQL

In [8]:
### Generate Data using FQL
formulas = {'os_top_hldr_id':'OS_TOP_HLDR_ID(5,#DATE,,M,,M)'}
df = qe.calculate(universe=id_univ, dates = time_series, data_dict=formulas,source = "FqlExpression",is_array=True)
df.data

,DATE,UNIVERSE,os_top_hldr_id
0,20230731,NVDA-US,NaN
1,20230731,AAPL-US,NaN
2,20230731,IVV-US,NaN
3,20230831,NVDA-US,NaN
4,20230831,AAPL-US,NaN
5,20230831,IVV-US,NaN


### Mixed FQL with Screening

In [9]:
formulas = [qe.QuantScreeningExpression(expr = 'P_PRICE(0,USD)',name= 'price',source= 'ScreeningExpression'),
            qe.QuantFqlExpression(expr = 'OS_TOP_HLDR_ID(5,#DATE,,M,,M)',name = 'top_hldrs',source='FqlExpression',is_array_return_type=True)]
df = qe.calculate(universe=id_univ, dates = time_series, formulas=formulas)
df.data

,DATE,UNIVERSE,price,top_hldrs
0,20230731,NVDA-US,467.29,"[M4004630, M4004543, M4001597, M4006564, M4032..."
1,20230731,AAPL-US,196.45,"[M4004630, M4004543, M4001597, M4006564, M4032..."
2,20230731,IVV-US,460.18,"[M4052193, M4250148, M4137677, M4178207, M4178..."
3,20230831,NVDA-US,493.55,"[M4004630, M4004543, M4001597, M4006564, M4032..."
4,20230831,AAPL-US,187.87,"[M4004630, M4004543, M4001597, M4032427, M4006..."
5,20230831,IVV-US,452.69,"[M4052193, M4250148, M4137677, M4178207, M4178..."
